In [1]:
import pandas as pd
import numpy as np

In [2]:
clima_df = pd.read_json('./dato_servicio_web.json')
clima_df['dloc'] = pd.to_datetime(clima_df['dloc'])

Validacion del tipo de datos de las variables

In [3]:
clima_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9852 entries, 0 to 9851
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   cc        9852 non-null   float64       
 1   desciel   9852 non-null   object        
 2   dh        9852 non-null   int64         
 3   dirvienc  9852 non-null   object        
 4   dirvieng  9852 non-null   int64         
 5   dloc      9852 non-null   datetime64[ns]
 6   ides      9852 non-null   int64         
 7   idmun     9852 non-null   int64         
 8   lat       9852 non-null   float64       
 9   lon       9852 non-null   float64       
 10  ndia      9852 non-null   int64         
 11  nes       9852 non-null   object        
 12  nmun      9852 non-null   object        
 13  prec      9852 non-null   float64       
 14  probprec  9852 non-null   int64         
 15  raf       9852 non-null   float64       
 16  tmax      9852 non-null   float64       
 17  tmin      9852

In [4]:
clima_df.head()

,cc,desciel,dh,dirvienc,dirvieng,dloc,ides,idmun,lat,lon,ndia,nes,nmun,prec,probprec,raf,tmax,tmin,velvien
0,78.96,Cielo nublado,6,Este,90,2024-10-29,20,54,17.3884,-97.2290,0,Oaxaca,Magdalena Zahuatlán,7.7,90,8.3,21.9,10.5,5.0
1,86.18,Cielo nublado,6,Noreste,45,2024-10-30,20,54,17.3884,-97.2290,1,Oaxaca,Magdalena Zahuatlán,5.4,50,11.2,22.4,11.8,6.3
2,78.79,Medio nublado,6,Noreste,45,2024-10-31,20,54,17.3884,-97.2290,2,Oaxaca,Magdalena Zahuatlán,5.4,50,15.5,22.8,10.9,9.0
3,77.47,Medio nublado,6,Noreste,45,2024-11-01,20,54,17.3884,-97.2290,3,Oaxaca,Magdalena Zahuatlán,0.0,0,15.5,21.2,10.8,9.1
4,76.97,Medio nublado,6,Noreste,45,2024-10-29,20,61,16.3672,-96.6368,0,Oaxaca,Monjas,2.9,0,13.1,23.7,13.0,4.2


In [5]:
def validar_dependencia_funcional(df, columnas_izquierda, n_variables):
    valores_previos = {}
    for index, row in df.iterrows():
        key = tuple(row[col] for col in columnas_izquierda)
        if key not in valores_previos:
            valores_previos[key] = {var: {'v': row[var], 'i': [index]} for var in n_variables}
        else:
            for var in n_variables:
                if valores_previos[key][var]['v'] != row[var]:
                    valores_previos[key][var]['i'].append(index)
    return valores_previos
def hayDependenciaFuncional(valores_previos, df, columnas_izquierda, n_variables, log=True):
    for _, value in valores_previos.items():
        for var in n_variables:
            if len(value[var]['i']) > 1:
                if log:
                    idx1, idx2 = value[var]['i'][0], value[var]['i'][1]
                    print(f"Filas {idx1} y {idx2} no cumplen con la dependencia funcional para la variable {var}:")
                    print(df.loc[idx1, columnas_izquierda + n_variables])
                    print(df.loc[idx2, columnas_izquierda + n_variables])
                return False
    return True

# Validacion de ciertas dependencias funcionales

In [6]:
cols_izquierda = ['ides']
cols_derecha = ['nes']
valores_previos = validar_dependencia_funcional(clima_df, cols_izquierda, cols_derecha)
hayDP = hayDependenciaFuncional(valores_previos, clima_df, cols_izquierda, cols_derecha, log=False)
if hayDP:
    print(f"({','.join(cols_izquierda)}) -> ({','.join(cols_derecha)})")

(ides) -> (nes)


In [7]:
cols_izquierda = ['ides']
cols_derecha = ['nes','dh']
valores_previos = validar_dependencia_funcional(clima_df, cols_izquierda, cols_derecha)
hayDP = hayDependenciaFuncional(valores_previos, clima_df, cols_izquierda, cols_derecha, log=True)
if hayDP:
    print(f"({','.join(cols_izquierda)}) -> ({','.join(cols_derecha)})")
else:
    print(f"({','.join(cols_izquierda)}) no determina a ({','.join(cols_derecha)})")

Filas 1220 y 9668 no cumplen con la dependencia funcional para la variable dh:
ides            19
nes     Nuevo León
dh               6
Name: 1220, dtype: object
ides            19
nes     Nuevo León
dh               5
Name: 9668, dtype: object
(ides) no determina a (nes,dh)


In [8]:
cols_izquierda = ['ides','idmun']
cols_derecha = ['lat','lon','nmun','dh']
valores_previos = validar_dependencia_funcional(clima_df, cols_izquierda, cols_derecha)
hayDP = hayDependenciaFuncional(valores_previos, clima_df, cols_izquierda, cols_derecha, log=False)
if hayDP:
    print(f"({','.join(cols_izquierda)}) -> ({','.join(cols_derecha)})")

(ides,idmun) -> (lat,lon,nmun,dh)


In [9]:
v,c = np.unique(clima_df['dloc'], return_counts=True)
print(len(v))
print(v)

4
['2024-10-29T00:00:00.000000000' '2024-10-30T00:00:00.000000000'
 '2024-10-31T00:00:00.000000000' '2024-11-01T00:00:00.000000000']


In [10]:
x = clima_df.groupby(by=['ides','idmun','lat','lon','nmun','dh']).size().reset_index(name='count')
print(x)

      ides  idmun      lat       lon                   nmun  dh  count
0        1      1  21.8798 -102.2960         Aguascalientes   6      4
1        1      2  22.2383 -102.0893               Asientos   6      4
2        1      3  21.8469 -102.7188               Calvillo   6      4
3        1      4  22.3664 -102.3000                  Cosío   6      4
4        1      5  21.9613 -102.3434            Jesús María   6      4
...    ...    ...      ...       ...                    ...  ..    ...
2458    32     54  22.3571 -101.7126          Villa Hidalgo   6      4
2459    32     55  22.3543 -102.8837             Villanueva   6      4
2460    32     56  22.7761 -102.5718              Zacatecas   6      4
2461    32     57  22.7354 -102.3660               Trancoso   6      4
2462    32     58  21.5122 -103.4070  Santa María de la Paz   6      4

[2463 rows x 7 columns]


Del anterior resultados podemos decir que hay una medición por cada municipio al dia. Por consiguiente hay cuatro registros que corresponden al dia actual, al siguiente, dos y tres días después

### Datos de la entidad predicción

In [11]:
datos = list(set(clima_df.columns) - set(['dloc', 'ides', 'idmun', 'lat', 'lon', 'nmun', 'dh', 'nes']))
print(datos)

['tmax', 'tmin', 'velvien', 'dirvieng', 'probprec', 'prec', 'desciel', 'dirvienc', 'raf', 'cc', 'ndia']


Validación de constrains, valores negativos, cotas máximas, precision de los valores numéricos(cantidad de decimales)

In [12]:
clima_df.describe()

,cc,dh,dirvieng,dloc,ides,idmun,lat,lon,ndia,prec,probprec,raf,tmax,tmin,velvien
count,9852.000000,9852.000000,9852.000000,9852,9852.000000,9852.000000,9852.000000,9852.000000,9852.000000,9852.000000,9852.000000,9852.000000,9852.000000,9852.000000,9852.000000
mean,69.486616,6.060495,105.598356,2024-10-30 12:00:00,19.236297,108.857085,20.023722,-98.743229,1.500000,3.182481,22.536541,16.447320,26.650142,14.765987,6.715032
min,0.070000,5.000000,0.000000,2024-10-29 00:00:00,1.000000,1.000000,14.679500,-117.054500,0.000000,0.000000,0.000000,2.900000,16.800000,2.400000,1.300000
25%,59.337500,6.000000,45.000000,2024-10-29 18:00:00,14.000000,23.000000,17.610900,-100.725200,0.750000,0.100000,0.000000,9.000000,23.800000,11.500000,4.600000
50%,75.940000,6.000000,90.000000,2024-10-30 12:00:00,20.000000,56.000000,19.327400,-98.233700,1.500000,1.600000,0.000000,12.600000,26.300000,13.700000,5.700000
75%,85.720000,6.000000,180.000000,2024-10-31 06:00:00,24.000000,128.000000,20.933000,-96.780600,2.250000,5.100000,50.000000,19.800000,29.500000,18.100000,7.500000
max,95.830000,7.000000,315.000000,2024-11-01 00:00:00,32.000000,570.000000,32.641200,-86.746100,3.000000,48.700000,90.000000,77.500000,37.100000,26.700000,40.700000
std,21.328516,0.284964,92.907719,NaN,7.356090,132.393580,3.340896,4.370896,1.118091,4.160717,37.015008,10.685497,3.658959,4.336232,3.866317
